In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install --q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq
from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.3.4 requires langchain-community<0.4.0,>=0.3.0, but you have langchain-community 0.0.29 which is incompatible.
langchain-experimental 0.3.4 requires langchain-core<0.4.0,>=0.3.28, but you have langchain-core 0.1.53 which is incompatible.
langchain-google-genai 2.0.6 requires langchain-core<0.4,>=0.3.15, but you have langchain-core 0.1.53 which is incompatible.
langchain-google-vertexai 2.0.8 requires langchain-core<0.4,>=0.3.15, but you have langchain-core 0.1.53 which is incompatible.
langchain-google-vertexai 2.0.8 requires pydantic<2.10,>=2.9, but you have pydantic 2.10.6 which is incompatible.
langchain-huggingface 0.1.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.1.53 which is incompatible.
langchain-ollama 0.2.2 requires langchain-core<0.4.0,>=0.3.27, but yo

In [30]:
llm=ChatGroq(
    model_name="llama-3.3-70b-versatile",
    api_key="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")

In [5]:
# from langchain_community.llms import Ollama

# base_model = "deepseek-r1"
# ollama_api = "http://localhost:11434"

# llm_local = Ollama(model=base_model, base_url=ollama_api)

In [31]:
import os
os.environ["SERPER_API_KEY"] = '18415b0324d1c6a9f71cb501e51ad7355c7cb2a63441bf5066958c0b88a44dae'

In [32]:
sales_rep_agent = Agent(
    llm=llm,
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [33]:
lead_sales_rep_agent = Agent(
    llm=llm,
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [34]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [35]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [36]:
from crewai_tools import BaseTool

In [37]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [38]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [39]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [40]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [41]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2
)

2025-01-28 21:28:16,506 - 23328 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [42]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)


 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: To conduct an in-depth analysis of DeepLearningAI, I first need to see what information is available in the directory that might be relevant to my task. This could include files containing company data, industry reports, or previous interactions with the company.

Action: List files in directory
Action Input: {} 

File paths: 
-./instructions/enterprise_solutions_framework.md
- ./instructions/sma

In [43]:
from IPython.display import Markdown
Markdown(result)


Subject: Congratulations on Your Recent Achievements and Exploring Future Growth Opportunities

Dear Andrew Ng,

I hope this message finds you well. As the CEO of DeepLearningAI, you've been at the forefront of innovation in the online learning platform sector. I wanted to take a moment to congratulate you on your recent achievements and express my admiration for the impact your company has had on the industry.

Our team has been following your progress, and we're excited about the potential opportunities for collaboration. We believe our solutions can support your goals and provide innovative learning platforms that can scale with your business. Our focus is on delivering cutting-edge technology that can help you achieve your vision for the future of online learning.

I'd love to schedule a call to discuss how our solutions can align with your company's objectives and explore potential areas for value addition. Our team is committed to providing support and resources tailored to the unique challenges faced by online learning platforms.

Please let me know if you're available for a call this week or next. I've included some information about our solutions below.

Best regards,
[Your Name]

---

Subject: Empowering Innovation and Scalability in Online Learning

Dear Andrew Ng,

As a leader in the online learning platform sector, you understand the importance of innovation and scalability. Our solutions are designed to support your goals and provide the necessary tools to take your business to the next level.

We've developed cutting-edge technology that can help you achieve your vision for the future of online learning. Our team is committed to delivering exceptional support and resources to ensure a seamless integration of our solutions into your existing infrastructure.

I'd be delighted to schedule a call to discuss how our solutions can address your company's specific needs and challenges. Please let me know if you're available this week or next.

Best regards,
[Your Name]

---

Subject: Strategic Partnership for Online Learning Excellence

Dear Andrew Ng,

As the CEO of DeepLearningAI, you're committed to delivering exceptional online learning experiences. We share your passion for innovation and excellence, and we believe our solutions can support your goals.

Our team is dedicated to providing strategic support and resources to help you overcome the unique challenges faced by online learning platforms. We're excited about the potential opportunities for collaboration and exploring ways to drive growth and innovation in the industry.

I'd love to schedule a call to discuss how our solutions can align with your company's objectives and explore potential areas for value addition. Please let me know if you're available this week or next.

Best regards,
[Your Name]